In [23]:
%pip install datasets


In [24]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [25]:
%pip install peft


Note: you may need to restart the kernel to use updated packages.


In [26]:
import time
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import os

In [5]:
!conda install pandas -y


/bin/bash: line 1: conda: command not found


In [27]:
from datasets import DatasetDict, Dataset

# Load your CSV files into Pandas DataFrames
train_df = pd.read_csv("C:/Users/umesh.malviya1/Downloads/train.csv")
test_df = pd.read_csv("C:/Users/umesh.malviya1/Downloads/test.csv")
validation_df = pd.read_csv("C:/Users/umesh.malviya1/Downloads/validation.csv")

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Create a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})

# Print dataset information
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})


In [ ]:
pip install --upgrade transformers


In [28]:
import torch
# Load pre-trained model and tokenizer
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)





In [29]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [30]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

In [31]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map: 100%|██████████| 500/500 [00:00<00:00, 855.86 examples/s]


In [32]:
#To save some time, we will subsample the dataset:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 2600 == 0, with_indices=True)#

Filter: 100%|██████████| 500/500 [00:00<00:00, 1814.74 examples/s]


In [33]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (5, 2)
Validation: (1, 2)
Test: (1, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1
    })
})


In [34]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [35]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [36]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [37]:
#output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
output_dir = "c:/Users/umesh.malviya1/Downloads/FINE_TUNNED_MODEL_WITH_PEFT"
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1 ,
    dataloader_num_workers=2,  # Set this value to a number greater than 0
    dataloader_prefetch_factor=2
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [38]:
peft_trainer.train()

peft_model_path = "c:/Users/umesh.malviya1/Downloads/FINE_TUNNED_MODEL_WITH_PEFT"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)


100%|██████████| 1/1 [54:16<00:00, 3256.23s/it]


{'loss': 49.25, 'grad_norm': 12.9375, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 3256.0763, 'train_samples_per_second': 0.002, 'train_steps_per_second': 0.0, 'train_loss': 49.25, 'epoch': 1.0}


('c:/Users/umesh.malviya1/Downloads/FINE_TUNNED_MODEL_WITH_PEFT\\tokenizer_config.json',
 'c:/Users/umesh.malviya1/Downloads/FINE_TUNNED_MODEL_WITH_PEFT\\special_tokens_map.json',
 'c:/Users/umesh.malviya1/Downloads/FINE_TUNNED_MODEL_WITH_PEFT\\tokenizer.json')